In [5]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
from datetime import datetime
from collections import Counter
from itertools import permutations

from tqdm import tqdm

import time
import pymongo
import bson
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [8]:
year_collector = []
for i in db_client["tweets_pipeline_v2"].find({"status": "pipelined"}):
    if "tweet_timestamp" in i:
        year_collector.append(i["tweet_timestamp"].year)
Counter(year_collector)

Counter({2020: 2506,
         2016: 196,
         2017: 231,
         2019: 224,
         2018: 238,
         2015: 169,
         2014: 94,
         2012: 4,
         2013: 15})

In [7]:
def to_reserved(_id):    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"reserved"},
                                    })

In [8]:
while True:
    db = db_client["tweets_pipeline_v2"].find({"status": "pipelined"})
    print("TOTAL:",db.count())
    counter = 0
    for i in db:
        if "tweet_timestamp" in i and i["tweet_timestamp"].year < 2020:
            _id = str(i["_id"])
            to_reserved(_id)
            counter+=1
    print("SEND TO RESERVE:", counter)        
    print()
    time.sleep(120)    

TOTAL: 5048
SEND TO RESERVE: 2084



KeyboardInterrupt: 

In [10]:
from collections import Counter

In [20]:
results_dict = {}
statuses = ["pipelined", 'deleted', "validated", 'graphicone_search', 'graphicone_feed', 'deleted_from_analytics']
for status in statuses:
    souce_counter = []
    for i in db_client["tweets_pipeline_v2"].find({"status": status, "source": {"$exists": True}}):
        souce_counter.append(i["source"])
    results_dict[status] = dict(Counter(souce_counter))

In [21]:
results_dict

{'pipelined': {'twitter': 936, 'instagram': 18, 'bing': 985},
 'deleted': {'twitter': 18389, 'google': 687, 'pdf': 1, 'instagram': 13},
 'validated': {'twitter': 422},
 'graphicone_search': {'google': 16, 'twitter': 2581, 'instagram': 18},
 'graphicone_feed': {'twitter': 548, 'instagram': 12},
 'deleted_from_analytics': {'twitter': 334, 'pdf': 1, 'instagram': 3}}

In [1]:
def to_pipelined(_id):    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"pipelined"},
                                    })

In [13]:
while True:
    db = db_client["tweets_pipeline_v2"].find({"status": "reserved"})
    print("TOTAL:",db.count())
    counter = 0
    for i in db:
        if "tweet_timestamp" in i and i["tweet_timestamp"].year < 2020:
            _id = str(i["_id"])
            print(_id)
            break
    break
#             to_reserved(_id)
#             counter+=1
#     print("SEND TO RESERVE:", counter)        
#     print()
#     time.sleep(120)

TOTAL: 10966
5dd1225502b5db85cf30ce70


In [20]:
db = db_client["tweets_pipeline_v2"].find({"status": "reserved"})
print("TOTAL:",db.count())
counter = 0
for i in db:
    if "tweet_timestamp" in i and i["tweet_timestamp"].year == 2019 and i["tweet_timestamp"].month == 10:
        _id = str(i["_id"])
        to_pipelined(_id)

TOTAL: 10881


In [90]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"status": "url-clone", 
                                               "grafeed_approver":{"$exists": True},
                                               "validator_username":{"$exists": True}
                                              }):
    counter +=1
#     if i["validator_username"] and i["grafeed_approver"]:
        
#         print(i)
#         break
print(counter)

0


In [89]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"status": "url-clone", 
                                               "grafeed_approver": {"$exists": True},
                                               "validator_username":{"$exists": True},
                                               'grafeed_confirmed': {"$exists": True},
                                              }):
    counter +=1
#     if i["validator_username"] and i["grafeed_approver"]:
        
#         print(i)
#         break
print(counter)

0


In [56]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"status": "url-clone", 
                                               "grafeed_approver": {"$exists": True},
                                               "validator_username":{"$exists": True},
                                               'grafeed_confirmed': {"$exists": True},
                                               "in_app":  {"$exists": True},
                                              }):
    counter +=1
#     if i["validator_username"] and i["grafeed_approver"]:
        
#     print(i)
#     break
print(counter)

0


In [87]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"status": "url-clone",
                                               "grafeed_approver": {"$exists": True},
                                               'grafeed_confirmed': {"$exists": True},
#                                                "$or": [{"grafeed_approver": {"$exists": True}}, 
# #                                                        {"validator_username": {"$exists": True}},
#                                                        {'grafeed_confirmed': {"$exists": True}},            
#                                                        ]
                                              }):
    counter += 1
#     to_search(i["_id"])
#     print(i)
#     print()
counter

0

In [88]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"status": "url-clone",
                                               "grafeed_approver": {"$exists": True},
                                               'grafeed_confirmed': {"$exists": False},
#                                                'grafeed_confirmed': {"$exists": True},
#                                                "$or": [{"grafeed_approver": {"$exists": True}}, 
# #                                                        {"validator_username": {"$exists": True}},
#                                                        {'grafeed_confirmed': {"$exists": True}},            
#                                                        ]
                                              }):
    counter += 1
#     to_deleted_analytics(i["_id"])
#     print(i)
#     print()
counter

0

In [77]:
def to_deleted_analytics(_id):    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"deleted_from_analytics",
                                            "recovering": True},
                                    })

In [83]:
def to_search(_id):    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"graphicone_search",
                                            "recovering": True},
                                    })

In [45]:
def to_feed(_id):    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"graphicone_feed"},
                                    })

In [47]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"status": "url-clone",
                                               "in_app":  {"$exists": True},
                                               "$or": [{"grafeed_approver": {"$exists": True}}, 
#                                                        {"validator_username": {"$exists": True}},
                                                       {'grafeed_confirmed': {"$exists": True}},
                                                       {"in_app":  {"$exists": True},}
                                                       ]}):
    counter += 1
#     if i["in_app"] == "search":
#         to_search(i["_id"])
# #         print(i["in_app"])
#     if i["in_app"] == "feed":        
#         to_feed(i["_id"])
        
#         print(i["in_app"])        
#     print()
counter

0

In [112]:
def to_deleted(_id):    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"deleted",
                                            "recovering": True},
                                    })

In [106]:
def to_validated(_id):    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"validated",
                                            "recovering": True},
                                    })

In [120]:
counter = 0
collector = []
for i in db_client["tweets_pipeline_v2"].find({"status": "url-clone",
                                               "validated_timestamp": {"$exists": True},
                                               "validator_username": {"$exists": True},
                                               "confirmed_after_validate":{"$exists": False},
#                                                "$or": [
# #                                                       {"grafeed_approver": {"$exists": True}}, 
#                                                        {"validator_username": {"$exists": True}},
# #                                                        {'grafeed_confirmed': {"$exists": True}},
# #                                                        {"in_app":  {"$exists": True},}
#                                                        ]
                                              }):
    counter += 1
    to_deleted(i["_id"])
#     to_validated(i["_id"])
#     print(i)
#     if "validated_timestamp" in i:
#         print(i["validated_timestamp"])
#     print()
counter

0

In [118]:
Counter(collector)

Counter({'_id': 1157,
         'tweet_text': 1156,
         'hashtags': 1156,
         'tweet_timestamp': 1156,
         'username': 1156,
         'user_id': 85,
         'user_counters': 85,
         'tweet_counters': 85,
         'img_urls': 1157,
         'tweet_id': 1156,
         'downloaded': 85,
         'download_error': 85,
         'img_type': 1157,
         'graph_type': 85,
         'status': 1157,
         'validated_timestamp': 1157,
         'validator_username': 1157,
         'imagehash': 61,
         'technical': 6,
         'dhash': 1155,
         'phash': 68,
         'duplicate_sort': 634,
         'extracted_text': 1152,
         'trasher_validated_labels': 16,
         'validated_for_trasher': 16,
         's3_url': 79,
         'confirmed': 7,
         'rejected': 7,
         'bert_tags': 1139,
         'bert_ts': 4,
         'pipelined_ts': 1139,
         'aws_answer': 134,
         'entity_prepared': 443,
         'GutenTagTrash': 63,
         'reason': 3,
  